# Test Sphere attraction - 

### Interacting spheres, one expects Birkhoff, and I think that is what happens.


In [1]:
import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt
%load_ext Cython 




## Force calc
### get netX, nety, netz, then force between them is m1*m2/d^2, to get vectors, we use netx/dist to get x contribution, etc.

In [21]:
%%cython --annotate
import numpy as np
import time
cimport cython
cimport numpy as cnp
cnp.import_array()

from libc.math cimport sqrt




cdef double oneOverroot3 = 0.57735026918962576450914878050195
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef void calcForce3(double d,  double s1x, double s1y, double s1z, double s1m, double s2x, double s2y, double s2z, double s2m, double cellSize, cnp.ndarray[double] forceSum):
    cdef double netx =  d*oneOverroot3 - s1x + s2x
    cdef double nety =  d*oneOverroot3 - s1y + s2y
    cdef double netz =  d*oneOverroot3 - s1z + s2z
    cdef double massF = s1m*s2m
    cdef double dist = sqrt(netx*netx + nety*nety + netz*netz)
    # from running this program, we know that once two cells start to overlap, the force goes 
    # down as dist goes to zero. This cellSize*cellSize/dist accomplishes that.
    if dist < cellSize:
        dist = cellSize*cellSize/dist# [0, 0, 0] #hack to not explode - ignore close points also try dist = cellSize #
    cdef double dC = dist*dist*dist # we need cubed, one to normalize the vector, the other two to do Newton's force
    cdef double factor = massF/dC
    forceSum[0] += factor*netx
    forceSum[1] += factor*nety
    forceSum[2] += factor*netz

#     # mass function
#     # mass density in my model is slightly altered from the one in the Katz paper.
#     # start with equation 3, take derivative, then get '
#     # https://www.wolframalpha.com/input?i=derivative+of+M+-+M%2F%281+%2B+2x%2FM%29+
#     # then divide by area of a sphere to get my mass density. 
#     # https://www.wolframalpha.com/input?i=%282M%5E2%2F%28M+%2B+2x%29%5E2%29%2F%284*pi*x%5E2%29
#     # then we have
#     # total mass is m, total radius is rS, so each particle we have represents 1/(4/3pirS^3) worth of volume. 
#     # we take the mass density (say a constant or by formula) and mult by the volume to get the mass of each particle.

# cdef double massOfPointConstantDensity(double m, double volS, int numPoints):
#     cdef double volPerPoint = volS/(numPoints*numPoints*numPoints)
#     return m/volS*volPerPoint # constant density, 

# return mass density at r * volperpoint
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef double massOfPointLocalEnergy(double r, double m, double volS, int numPoints):
    cdef double volPerPoint = volS/(numPoints*numPoints*numPoints)
    return m*m/(2*np.pi*r*r*(m + 2*r)*(m + 2*r))*volPerPoint


@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef long doTheWork():
    # idea - make two spheres, each with points that vary in mass with radius
    # mass points are on non interecting planes of z 
    # masses are held apart in x direction, moved together.
    # both spheres have same number of points, just masses of each point varies 
    # points in an XY plane are laid out in grid, with no points at r > r of sphere 
    # (keeping in mind where each sphere has an origin at: 0 0 0 
    
    cdef long rS1 = 1000 # size of spheres
    cdef long rS2 = 1000 # size of spheres
    cdef double volS1 = (4/3*np.pi*rS1*rS1*rS1)
    cdef double volS2 = (4/3*np.pi*rS2*rS2*rS2)
    cdef long numPoints = 10 # (2*numPoints)**3 points in the sphere use even numbers only so as to not get an infinite mass density...
    cdef double m1 = 10
    cdef double m2 = 10
    cdef double cellSize1 = rS1/(2*numPoints)
    cdef double cellSize2 = rS2/(2*numPoints)
    # keep min dist and max dist odd weird numbers so points dont hit each other
    cdef double minDist = 400.12129876523
    cdef double maxDist = 1500.2212123123
    cdef long numDistPoints = 200

    cdef long maxNumberOfMassPoints = (numPoints)*(numPoints)*(numPoints)

    cdef cnp.ndarray[double] s1x = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s1y = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s1z = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s1m = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s2x = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s2y = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s2z = np.empty(maxNumberOfMassPoints, dtype=np.double)
    cdef cnp.ndarray[double] s2m = np.empty(maxNumberOfMassPoints, dtype=np.double)
    
    # make the masses as distributed points 
    cdef double com1X = 0;
    cdef double com1Y = 0;
    cdef double com1Z = 0;
    cdef double com2X = 0;
    cdef double com2Y = 0;
    cdef double com2Z = 0;
    cdef double totalMass1 = 0
    cdef double totalMass2 = 0
    cdef double r = 0
    cdef double x = 0
    cdef double y = 0
    cdef double z = 0
    cdef double m = 0
    cdef long np1 = 0
    for x in np.linspace(-rS1, rS1, numPoints):
        for y in np.linspace(-rS1, rS1, numPoints):
            for z in np.linspace(-rS1, rS1, numPoints):
                r = sqrt(x*x + y*y + z*z)
                if r < rS1:
                    #add a point to each
                    # mass 
                    m = massOfPointLocalEnergy(r, m1, volS1, numPoints)
                    totalMass1 += m
                    s1x[np1] = x
                    s1y[np1] = y
                    s1z[np1] = z
                    s1m[np1] = m
                    np1 += 1
                    com1X += m*x
                    com1Y += m*y
                    com1Z += m*z
                    
    cdef long np2 = 0
    for x in np.linspace(-rS2, rS2, numPoints):
        for y in np.linspace(-rS2, rS2, numPoints):
            for z in np.linspace(-rS2, rS2, numPoints):
                r = sqrt(x*x + y*y + z*z)
                if r < rS2:
                    #add a point to each
                    # mass                 
                    m = massOfPointLocalEnergy(r, m2, volS2, numPoints)
                    totalMass2 += m
                    s2x[np2] = x
                    s2y[np2] = y
                    s2z[np2] = z
                    s2m[np2] = m
                    np2 += 1
                    com2X += m*x
                    com2Y += m*y
                    com2Z += m*z
    
        
    print('centre of mass 1:',com1X, com1Y, com1Y)
    print('centre of mass 2:',com2X, com2Y, com2Y)
    print('total mass 1:',totalMass1)
    print('total mass 2:',totalMass2)
    print('MaxNumMassPoints:',maxNumberOfMassPoints)
    print('NumMassPoints:',np1, np2)
    
    # now we have our points in two numpy arrays, 
    # for each point, we want to figure out the forces and add them all up.

# Do the grunt work
    distances = np.linspace(minDist, maxDist, numDistPoints)
    cdef long numDistances = distances.size
    cdef cnp.ndarray[double] forcesX = cnp.ndarray(numDistances, dtype=np.double)
    cdef cnp.ndarray[double] forcesY = cnp.ndarray(numDistances, dtype=np.double)
    cdef cnp.ndarray[double] forcesZ = cnp.ndarray(numDistances, dtype=np.double)
    cdef double cellSize = 2*max(cellSize1, cellSize2)
    cdef long numMasses = s1x.size
    cdef cnp.ndarray[double] forceAtD = cnp.ndarray(3, dtype=np.double)
    cdef double s1xn = 0
    cdef double s1yn = 0
    cdef double s1zn = 0
    cdef double s1mn = 0
    cdef long distCount = 0
    cdef double d = 0
    cdef long n1 = 0
    cdef long n2 = 0
    for d in distances:
        forceAtD[0] = 0 # we sum into this little array
        forceAtD[1] = 0
        forceAtD[2] = 0
        for n1 in range(numMasses):
            s1xn = s1x[n1]
            s1yn = s1y[n1]
            s1zn = s1z[n1]
            s1mn = s1m[n1]
            for n2 in range(numMasses):
                calcForce3(d, s1xn, s1yn, s1zn, s1mn, s2x[n2], s2y[n2], s2z[n2], s2m[n2], cellSize, forceAtD)
        forcesX[distCount] = forceAtD[0]
        forcesY[distCount] = forceAtD[1]
        forcesZ[distCount] = forceAtD[2]
        distCount += 1

    # output use regular array (list)
    #cdef cnp.ndarray[double, ndim=2] 
    # cdef cnp.ndarray[double, ndim=2] forces = cnp.ndarray(dtype=np.double, ndim=2)
    # forces.append(forcesX)
    # forces.append(forcesY)
    # forces.append(forcesZ)
    
    # print('force distances:',len(forces))
    # print('forces per distance:',len(forces[0]))
    #print('force:',np.sum(forces[34][0]))
    # return forces
    return numMasses*numMasses*len(distances)


start_time = time.time()
numCalcs = doTheWork()
millionsPerSec = (numCalcs/1e6)/(time.time() - start_time)
print("dotheWork took:", (time.time() - start_time))
print("million per sec: ", millionsPerSec)
print("numCalcs: ", numCalcs)



centre of mass 1: -2.0651618379716606e-15 -2.2753079689292474e-15 -2.2753079689292474e-15
centre of mass 2: -2.0651618379716606e-15 -2.2753079689292474e-15 -2.2753079689292474e-15
total mass 1: 0.13679628237682248
total mass 2: 0.13679628237682248
MaxNumMassPoints: 1000
NumMassPoints: 360 360
dotheWork took: 10.834882020950317
million per sec:  18.458901279871828
numCalcs:  200000000


In [3]:

#forces = doTheWork()

xForce = []
yForce = []
zForce = []
totalForce = []
maxForce = 0
minForce = 1e99
maxForceX = 0
minForceX = 1e99
maxForceZ = 0
minForceZ = 1e99
for xyzNet, d in zip(forces, distances):
    d2 = d*d
    dM = d2 # set to d2 to see errors from r^2, set dM to 1 to see actual r^2 graph
    #dM = 1
    # Sum along columns (axis=0) - get total x, y, z forces, which 
    # xyzNet = np.sum(fV, axis=0) 
    xT = xyzNet[0]*dM
    yT = xyzNet[1]*dM
    zT = xyzNet[2]*dM
    tF = np.sqrt(xT*xT + yT*yT + zT*zT)
    totalForce.append(tF)
    xForce.append(xT)
    yForce.append(yT)
    zForce.append(zT)
    maxForce = max(maxForce, tF)
    minForce = min(minForce, tF)
    maxForceX = max(maxForceX, xT)
    minForceX = min(minForceX, xT)
    maxForceZ = max(maxForceZ, zT)
    minForceZ = min(minForceZ, zT)
    
np.savetxt('forces.csv', totalForce, delimiter=',')

if np.abs(minForce - maxForce)/maxForce < 0.001:
    minForce = 0.99999999999999*maxForce
    maxForce = 1.00000000000001*maxForce

if np.abs(minForceX - maxForceX)/maxForce < 0.001:
    minForceX = 0.99999999999999*maxForceX
    maxForceX = 1.00000000000001*maxForceX

if np.abs(minForceZ - maxForceZ)/maxForce < 0.001:
    minForceZ = 0.99999999999999*maxForceZ
    maxForceZ = 1.00000000000001*maxForceZ


print('totalForce0:',totalForce[0])
print('totalForce40:',totalForce[40])

fig = plt.figure(figsize=(7, 5), dpi=200)
plt.plot(distances, totalForce, marker="none", color='blue', lw=1.5, label='Total Force')
plt.xlabel('distance');
plt.ylabel('force total');
plt.legend(loc='lower right')
plt.axis((0, maxDist, minForce, maxForce));
#plt.savefig('../chapters/images/force.png', bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(7, 5), dpi=200)
plt.plot(distances, xForce, marker="none", color='green', lw=1.5, label='Force')
plt.xlabel('distance');
plt.ylabel('force x');
plt.legend(loc='lower right')
plt.axis((0, maxDist, minForceX, maxForceX));
#plt.savefig('../chapters/images/force.png', bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(7, 5), dpi=200)
plt.plot(distances, yForce, marker="none", color='yellow', lw=1.5, label='ForceZ')
plt.xlabel('distance');
plt.ylabel('force z');
plt.legend(loc='lower right')
plt.axis((0, maxDist, minForceZ, maxForceZ));
#plt.savefig('../chapters/images/force.png', bbox_inches='tight')
plt.show()

NameError: name 'forces' is not defined